# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install requirements

In [2]:
!pip install pytorch_lightning

     |████████████████████████████████| 527 kB 5.2 MB/s 
     |████████████████████████████████| 133 kB 59.6 MB/s 
     |████████████████████████████████| 397 kB 56.6 MB/s 
     |████████████████████████████████| 829 kB 43.6 MB/s 
     |████████████████████████████████| 952 kB 38.4 MB/s 
     |████████████████████████████████| 596 kB 56.9 MB/s 
     |████████████████████████████████| 1.1 MB 48.4 MB/s 
     |████████████████████████████████| 271 kB 47.7 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 144 kB 54.4 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=fefe4d598a620e3cafe3600da4cc804638d5af8cf6bd437648c8e01c06b97f1a
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successful

# Import relevant libraries

In [3]:
import sys
import os
module_path = os.path.join(os.path.abspath(''), "drive/MyDrive/Master_Thesis_Profactor/zdmp/")
if module_path not in sys.path:
  sys.path.append(module_path)

import zdmp
from utils.lightning_classifier import Classifier

import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader

# Train Luna Model

In [4]:

def save_model(model, name):
    path = os.path.join(
        "/content/drive/MyDrive/Master_Thesis_Profactor/zdmp/pretrained_weights",
        f"{name}.pth.tar"
    )
    torch.save(model.state_dict(), path)

In [5]:
# Make the results reproducible
pl.seed_everything(42)

# Prepare dataset
ds_train = zdmp.get_train_data()
ds_test = zdmp.get_test_data()

# Prepare Dataloaders
dl_train = DataLoader(ds_train, batch_size=32, num_workers=2, shuffle=True)
dl_test = DataLoader(ds_test, batch_size=32, num_workers=2)

# Prepare model
model = zdmp.get_luna(pretrained=False)

# Prepare Classifier to train the model
classifier = Classifier(model)

# Prepare the Trainer
trainer = pl.Trainer(gpus=1, precision=16, auto_lr_find=True)

# Find the best learning rate
trainer.tune(classifier, train_dataloaders=dl_train, val_dataloaders=dl_test)

# Train
trainer.fit(classifier, dl_train, dl_test)

# Save the trained weights
save_model(classifier.model, "luna")

Global seed set to 42


MisconfigurationException: ignored

In [7]:
!tensorboard dev upload --logdir '/content/lightning_logs'


***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

/content/lightning_logs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) y

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&state=x4axnRSN1sH02EB7UxnagUuO8uUBku&prompt=consent&access_type

In [ ]:
import importlib

importlib.reload(zdmp)
importlib.reload(utils.lightning_classifier)

NameError: ignored